# Understanding Monads using Python

Many tutorials I've seen have tended to focus on a top-down approach of explanation. I'd like to try a bottom-up approach, using just the normal plain old Python you already know.

We're going to make a `Result` monad, to encapsulate both success and failure states, and we're going to compare this to using plain old exceptions.

## It's all about the bind

Imagine you did a sequence of operations like this:

In [22]:
s = "    This is my wonderful long string   "
output: int = (
    s.strip()
    .replace("w", "W")
    .split(" ")
    .__len__()
)
print(output)

6


Look carefully at how the data, which is originally the string `s`, propagates through the chain of calls and the data gets modified along the way. Now imagine that every successive chained operation was wrapped in a method call called `bind()`. Without going into too much detail, let's just write out what the previous operation might look like. This code won't run, I just want you to visualize how the chain of operations is still being respected, albeit with extra parts:

```python
s = "    This is my wonderful long string   "
output = (
    s.bind(str.strip)
    .bind(lambda s: s.replace("w", "W"))
    .bind(lambda s: s.split(" "))
    .bind(lambda lst: len(lst))
)
print(output)
```

The `bind()` is just a way to apply a function to the object. For the above to have any chance of working, it is necessary that every usage of `bind()` returns another thing itself also has a `bind()` method, right? That is a bit annoying; however, on the plus side if we have a standard way of applying a succession of function calls to some data, that gives us the opportunity to make some decisions about what to do when a particular function needs to be applied.

Crucially, one of those choices is about what to do if a particular function being applied causes an error. This is the key idea. So to summarize:
- We need to wrap our data in some kind of object wrapper that has a `bind()` method
- The `bind()` method will execute a function, passing the current value (or "state") to that function
- It is important that the `bind()` method returns an object that also has a `bind()` method, allowing more chaining.

With all that out of the way, let's make a set of classes that will handle "success" and "failure".

## New classes to wrap our data, and implement `bind`

I'll write out the code first, and then we'll talk about it.

In [25]:
from dataclasses import dataclass
from abc import abstractmethod
from typing import Any

@dataclass
class Result:
    @abstractmethod
    def bind(self, f):
        raise NotImplementedError

@dataclass
class Ok(Result):
    value: Any
    def bind(self, f):
        try:
            return Ok(f(self.value))
        except Exception as e:
            return Err(e)

@dataclass
class Err(Result):
    value: Exception
    def bind(self, f):
        return self

We don't actually need the base class `Result`, but it will help with type annotations.

The import bit to explain is that we have two classes that will **wrap** our actual data. These two classes are `Ok` and `Err`. Pay attention to the following:
- They both have a `bind()` method
- For each, their `bind()` method returns an object that itself has a `bind()` method.

Let's talk about the last point for a minute.  The `Err` class is easy. It's internal `value` is always an exception type, and it's `bind()` method always just returns itself. This might seem pointless, but the true point is to satisfy the two requirements above.

The other class, `Ok`, is more interesting. Inside its `bind()` method, it will apply the given function `f` to its internal data, and if this call succeeds it will wrap that new result in a new instance of `Ok`, and return that. However, if the evaluation of `f(self.value)` fails, then it will wrap the resulting `Exception` type in a new instance of `Err` and return that instead.

The `Ok.bind()` method can only return `Ok` or `Err`. There are no other possible options. Since both `Ok` and `Err` satisfy our two requirements, we should be able to use our new classes.

## Using the `Ok` and `Err` classes

Let's try to implement the idealised example from earlier, this time with real code using our two examples.

In [16]:
s = "    This is my wonderful long string   "
output: Result = (
    Ok(s)
    .bind(str.strip)
    .bind(lambda s: s.replace("w", "W"))
    .bind(lambda s: s.split(" "))
    .bind(lambda lst: len(lst))
)
print(output)

Ok(value=6)


How interesting: instead of getting a `6`, as before, we get the value wrapped in an `Ok` object wrapper. That makes sense, because remember that every application of `bind()` will keep returning a new wrapper.  

Note the type annotation, `Result`, in the code snippet above. That's why we have a base class for `Ok` and `Err`, because it allows us to describe that an identifier's type, like `output` above, could be one of those two subclasses.

To get the data out, you can simply access the `value` attribute:

In [17]:
print(output.value)

6


So far this seems like a laborious way to do a very simple chaining operation. At least, in the original plain Python code it was quite simple. There were earlier implications that this strategy using `bind()` might be useful for error handling. With that in mind let's modify our example to introduce an error. 

## Compare error handling: exceptions vs bind

Here I've changed the value `s` from a string to a float. None of the string manipulations are going to work, and we're going to get an error:

In [18]:
s = 123.4
output = (
    s.strip()
    .replace("w", "W")
    .split(" ")
    .__len__()
)
print(output)

AttributeError: 'float' object has no attribute 'strip'

By changing the data type of the data, `s`, our code now blows up. In normal Python, the way to deal with this is with exception handling. Let's add some.

In [19]:
s = 123.4
try:
  output = (
    s.strip()
    .replace("w", "W")
    .split(" ")
    .__len__()
  )
except Exception as e:
    print(f"There was an error: {e}")
else:
    print(output)

There was an error: 'float' object has no attribute 'strip'


That's much less explosive!

Let's try to process the same bad data with our new code that uses `bind()` and the `Ok` and `Err` types. No exception handler will be added.

In [20]:
s = 123.4
output = (
    Ok(s)
    .bind(str.strip)
    .bind(lambda s: s.replace("w", "W"))
    .bind(lambda s: s.split(" "))
    .bind(lambda lst: len(lst))
)
print(output)

Err(value=TypeError("descriptor 'strip' for 'str' objects doesn't apply to a 'float' object"))


This is super interesting. We didn't get an exception raised, but what we observe is the the _output value itself_ is an instance of the `Err` class. Furthermore, the internal value of this object contains our exception. I personally find it quite interesting that the error message is better in this case than what we got with the more natural error handling approach with the non-bind code.

In the above example, the error was introduced right at the start. It might help to understand even more deeply by seeing what happens if we move the error to somewhere deeper in the call chain.

In [21]:
s = "    This is my wonderful long string   "
output = (
    Ok(s)
    .bind(str.strip)
    .bind(lambda s: s.replace("w", "W"))
    # This is new, I'm replacing the data in the stream with `None`
    .bind(lambda s: None) 
    # This application will fail because you can't call `split` on `None`
    .bind(lambda s: s.split(" "))
    .bind(lambda lst: len(lst))
)
print(output)

Err(value=AttributeError("'NoneType' object has no attribute 'split'"))


And there go you. The failure along the chain of applications got caught and propagated through.

Just as we saw with the success case, it was necessary to extract the value of the result, `6`. Likewise, it will be necessary here to extract the exception. If you wanted to handle both the success and the error case, you might use the `match` statement:

In [33]:
def some_processing(s: str|None) -> Result:
    return (
        Ok(s)
        .bind(str.strip)
        .bind(lambda s: s.replace("w", "W"))
        .bind(lambda s: s.split(" "))
        .bind(lambda lst: len(lst))
    )

First we'll try valid data:

In [30]:
s = "    This is my wonderful long string   "
match some_processing(s):
    case Ok(value):
        print(f"Success. The result was {value}")
    case Err(e):
        print(f"Failure. The exception was {e}")

Success. The result was 6


...and now invalid data:

In [32]:
s = None
match some_processing(s):
    case Ok(value):
        print(f"Success. The result was {value}")
    case Err(e):
        print(f"Failure. The exception was {e}")

Failure. The exception was descriptor 'strip' for 'str' objects doesn't apply to a 'NoneType' object


We made good use of the `match` statement's pattern matching features in extract the values from each of the `Ok` and `Err` variants.

## Conclusion

Perhaps a good way to compare the two approaches above for error handling is to describe them like this: 
- Exceptions jump directly out of the path of execution and race up the stack to the nearest exception handler where you can handle them appropriately.
- Monadic error handling, e.g. using the `Ok` and `Err` classes above, instead allows the chain of execution to _bypass_ code once an error condition has been found. Crucially, the error condition and the success condition are now both different types of data which means that their handling and detection can be managed with type-checking rather than an error handler.

Which is better? The classic question. My personal view is that programming languages that do a good job of modelling types, or said in another way have a rich type system, are likely to be better off with monadic styles. On the other hand, programming languages that focus less on the type system, for example dynamically-typed systems like Python and JavaScript, are likely to find little benefit using monadic error handling compared to exception handling.

One compelling advantage of the monadic approach for a language like Python, is that it allows the error types to be described by the type system. For example, I could conceive of the following kind of function signature based on using the earlier types described in this post:

```python
# This code won't work, it's for explanation only
def some_processing(a: str|None) -> Result[Ok[int], Err[Exception]]:
    ...
```